In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

Leer el archivo CSV que contiene los datos históricos de las pizzas

In [ ]:
df_pizzas = pd.read_csv(r'Datawarehouse\pizzas_normales.csv')

Agrupar los datos por fecha y sumar las unidades totales vendidas por mes

In [ ]:
pizzas_mes = df_pizzas.groupby('fecha')['unidades_total'].sum().reset_index()
pizzas_mes

Crear una instancia de `SingleTableMetadata` y detectar la estructura del dataframe `pizzas_mes`

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(pizzas_mes)

Crear una instancia de `CTGANSynthesizer` con los metadatos y entrenar el modelo con los datos históricos

In [ ]:
synthesizer = CTGANSynthesizer(metadata, epochs=1000)
synthesizer.fit(pizzas_mes)

Generar nuevos datos sintéticos para los periodos anteriores especificados

In [ ]:
fechas_completas = pd.date_range(start='2022-01-01', end='2022-12-01', freq='MS')
df_sintetico = synthesizer.sample(num_rows=len(fechas_completas))

# Asignar las fechas completas a los datos sintéticos
df_sintetico['fecha'] = fechas_completas

Convertir la columna 'fecha' a tipo datetime si aún no lo está en ambos dataframes y combinar los datos originales con los sintéticos

In [ ]:
pizzas_mes['fecha'] = pd.to_datetime(pizzas_mes['fecha'])
df_sintetico['fecha'] = pd.to_datetime(df_sintetico['fecha'])
df_combinado = pd.concat([pizzas_mes, df_sintetico]).sort_values('fecha').reset_index(drop=True)
df_combinado

Crear un gráfico de barras para visualizar las unidades totales vendidas por mes

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(df_combinado['fecha'], df_combinado['unidades_total'], color='skyblue')

# Personalizar el gráfico
plt.xlabel('Mes')
plt.ylabel('Unidades Totales')
plt.title('Unidades Totales por Mes')
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mejor visualización si es necesario

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Guardar el dataframe combinado en un nuevo archivo CSV

In [ ]:
df_combinado.to_csv(r'Datawarehouse\pizzas_normales_sd.csv', index=False)